In [ ]:
from pathlib import Path
from tqdm import tqdm
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torchvision import transforms

from leaf.dta import LeafDataset, GetPatches, TransformPatches, RandomGreen, LeafDataLoader
from leaf.model import LeafModel, train_one_epoch, validate_one_epoch, warmup
from torch.utils.data import RandomSampler, DataLoader

from torch.optim.lr_scheduler import CyclicLR

from torch.utils.tensorboard import SummaryWriter


# Transforms with normalizations for imagenet
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'warmup': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'patches': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [ ]:
save_dir = Path("/mnt/hdd/leaf-disease-outputs")
save_dir.mkdir(exist_ok=True)
logging_dir = Path("/mnt/hdd/leaf-disease-runs")
logging_dir.mkdir(exist_ok=True)

batch_size = 16
val_batch_size = 32

num_workers = 4

learning_rate = 1e-4
#min_learning_rate = 1e-6
weight_decay = 1e-6
warmup_lr = 1e-7

#T_0 = 10

patches_logging_steps = 1000
train_logging_steps = 250
save_checkpoints = True

train_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["train"])
patches_dset = LeafDataset("./data/patches_train", "./data/patches_train/labels.csv", extended_labels=True, transform=data_transforms["patches"])
warmup_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["warmup"])
val_dset = LeafDataset("./data/patches_val", "./data/patches_val/labels.csv", extended_labels=True, transform=data_transforms["val"])

train_dataloader = DataLoader(train_dset, batch_size=batch_size, sampler=RandomSampler(train_dset, replacement=True), num_workers=num_workers, pin_memory=True)
patches_dataloader = LeafDataLoader(patches_dset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = LeafDataLoader(val_dset, batch_size=val_batch_size, shuffle=False, num_workers=num_workers)

leaf_model = LeafModel("tf_efficientnet_b4_ns", model_prefix="MONEVE", save_dir=save_dir, logging_dir=logging_dir)

# optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=1, eta_min=min_learning_rate, last_epoch=-1)
leaf_model.update_optimizer_scheduler(optimizer, scheduler)

In [ ]:
leaf_model.load_checkpoint("MONEVE_epoch_2")

In [ ]:
optimizer = SGD(leaf_model.model.parameters(), lr=1e-3, weight_decay=weight_decay)
scheduler = CyclicLR(optimizer, 1e-6, 1e-3, step_size_up=2*len(train_dataloader), mode="exp_range", gamma=0.9)

In [ ]:
leaf_model.update_optimizer_scheduler(optimizer, scheduler)

In [ ]:
run_name = "RandomGreen-40-epochs"
tb_writer = SummaryWriter(log_dir=leaf_model.logging_dir / run_name)
for epoch in range(40):
    train_one_epoch(leaf_model, train_dataloader, log_steps=200, save_at_log_steps=save_checkpoints, epoch_name=f"{run_name}-{epoch}")
    val_loss, val_acc = validate_one_epoch(leaf_model, val_dataloader)
    leaf_model.log_validation(val_loss, val_acc, epoch, tb_writer)
    leaf_model.save_checkpoint(f"{run_name}-{epoch}", epoch=epoch)

In [ ]:
optimizer = SGD(leaf_model.model.parameters(), lr=1e-3, weight_decay=weight_decay)
scheduler = CyclicLR(optimizer, 1e-6, 1e-3, step_size_up=len(train_dataloader), mode="triangular2")

In [ ]:
n_steps = 4 * len(train_dataloader)
optimizer_lr_array = np.zeros((n_steps,))
scheduler_lr_array = np.zeros((n_steps,))

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
    
    def forward(self, X):
        return self.linear(X)

In [ ]:
net = MyNet().to(torch.device("cuda"))

In [ ]:
for i in tqdm(range(n_steps)):
    optimizer.zero_grad()
    data = torch.tensor([1.0]).reshape(1, 1).to(torch.device("cuda"))
    out = net.forward(data)
    out.backward()
    optimizer_lr_array[i] = optimizer.param_groups[0]["lr"]
    scheduler_lr_array[i] = scheduler.get_last_lr()[0]
    optimizer.step()
    scheduler.step()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(optimizer_lr_array)

In [ ]:
plt.plot(scheduler_lr_array)

In [ ]:
max(scheduler_lr_array)

In [ ]:
min(scheduler_lr_array)

In [ ]:
df = pd.DataFrame({"opt": optimizer_lr_array, "sched": scheduler_lr_array })

In [ ]:
df.iloc[-20:,:]

In [ ]:
(optimizer_lr_array == scheduler_lr_array).all()